# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import urllib.request
from azureml.core.model import Model


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-automl-experiment'

experiment=Experiment(workspace=ws, name=experiment_name)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-189146
aml-quickstarts-189146
southcentralus
5a4ab2ba-6c51-4805-8155-58759ad589d8


# Create or Attach an AmlCompute cluster

In [3]:

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-com-clst" #automl-com-clst #automl-com-clst

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


# Data

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
#key = "BankMarketing Dataset"
key="Maternal_Health_Risk_Data_Set" #"bankmarketing_train" #bankmarketing_train
#bankmarketing_train
description_text = "UCI machine Learning Maternal Health Risk Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
#https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
count,1014.000000,1014.000000,1014.000000,1014.000000,1011.000000,1014.000000
mean,29.871795,113.198225,76.460552,8.725986,98.665678,74.301775
std,13.474386,18.403913,13.885796,3.293532,1.373367,8.088702
min,10.000000,70.000000,49.000000,6.000000,98.000000,7.000000
25%,19.000000,100.000000,65.000000,6.900000,98.000000,70.000000
50%,26.000000,120.000000,80.000000,7.500000,98.000000,76.000000
75%,39.000000,120.000000,90.000000,8.000000,98.000000,80.000000
max,70.000000,160.000000,100.000000,19.000000,103.000000,90.000000


# Review the Dataset Result

In [5]:
dataset.take(5).to_pandas_dataframe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98,86,high risk
1,35,140,90,13.0,98,70,high risk
2,29,90,70,8.0,100,80,high risk
3,30,140,85,7.0,98,70,high risk
4,35,120,60,6.1,98,76,low risk


In [8]:
from sklearn.model_selection import train_test_split
#TRAIN_d, TEST_d = train_test_split(df, shuffle=True)
TRAIN_d, TEST_d = train_test_split(df, test_size=0.2, random_state=24, shuffle=True)
TRAIN_d.to_csv('train.csv',index = False)

ds_new = ws.get_default_datastore()
ds_new.upload_files(files = ['./train.csv'])
#ds_new.upload(src_dir='.',target_path='.')


train_data = Dataset.Tabular.from_delimited_files(path=[(ds_new,('train.csv'))])


"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

# Train
#### This creates a general AutoML settings object. Udacity notes: These inputs must match what was used when training in the portal. label_column_name has to be "RiskLevel" for example.

In [10]:
# TODO: Put your automl settings here
 # "primary_metric" : 'AUC_weighted'
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "n_cross_validations":3,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="RiskLevel",   
                             #path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )

In [11]:
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
remote_run = experiment.submit(config= automl_config, show_output= True)

Submitting remote run.
No run_configuration provided, running on automl-com-clst with default configuration
Running on remote compute: automl-com-clst


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl-experiment,AutoML_252116ab-f119-48b2-84ff-8dbf083b8815,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
      

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [13]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl-experiment,AutoML_252116ab-f119-48b2-84ff-8dbf083b8815,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+------------------------------+------------------------------+------------------------------+
|Column name          

{'runId': 'AutoML_252116ab-f119-48b2-84ff-8dbf083b8815',
 'target': 'automl-com-clst',
 'status': 'Completed',
 'startTimeUtc': '2022-03-16T20:53:07.630393Z',
 'endTimeUtc': '2022-03-16T21:15:34.251314Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'automl-com-clst',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-automl-experiment","subscription_id":"5a4ab2ba-6c51-4805-8155-58759ad589d8","resource_group":"aml-quickstarts-189146","workspace_name":"quick-starts-ws-189146","region":"southcentralus","compute_targ

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
#import pickle
#with open(best_model_output._path_on_datastore, "rb" ) as f:
#    best_model = pickle.load(f)
#best_model
automl_best_run, automl_fitted_model = remote_run.get_output()
print("===== AutoMl Best Run ========")
print(automl_best_run)
print("\n  ===== AutoMl Best Fitted Model ========")
print(automl_fitted_model)
automl_best_run.get_tags()
automl_best_run_metrics = automl_best_run.get_metrics()

for metric_name in automl_best_run_metrics:
    metric = automl_best_run_metrics[metric_name]
    print("\n",metric_name, metric)
    

#automl_best_run, automl_fitted_model = remote_run.get_output()
#print(hasattr(automl_fitted_model, 'steps'))
#print("\n")
#print(automl_best_run.get_metrics())
#print("\n")
#print(automl_best_run.get_tags())   

===== AutoMl Best Run ========
Run(Experiment: capstone-automl-experiment,
Id: AutoML_252116ab-f119-48b2-84ff-8dbf083b8815_37,
Type: azureml.scriptrun,
Status: Completed)

  ===== AutoMl Best Fitted Model ========
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False)), ('32', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.3, gamma=0, grow_policy='lossguide', max_bin=63, max_depth=7, max_leaves=0, n_estimators=25, n_jo

In [15]:
print(automl_fitted_model.steps)

[('datatransformer', DataTransformer(
    task='classification',
    is_onnx_compatible=True,
    enable_feature_sweeping=False,
    enable_dnn=False,
    force_text_dnn=False,
    feature_sweeping_timeout=86400,
    featurization_config=None,
    is_cross_validation=True,
    feature_sweeping_config={}
)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(
    estimators=[('0', Pipeline(
        memory=None,
        steps=[('maxabsscaler', MaxAbsScaler(
            copy=True
        )), ('lightgbmclassifier', LightGBMClassifier(
            min_data_in_leaf=20,
            random_state=None,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            )
        ))],
        verbose=False
    )), ('18', Pipeline(
        memory=None,
        steps=[('standardscalerwrapper', StandardScalerWrapper(
            copy=True,
            with_mean=False,
            with_std=False
        )), ('xg

In [ ]:
#TODO: Save the best model using Onnx
#from azureml.automl.runtime.onnx_convert import OnnxConverter

#automl_best_run, automl_fitted_model = remote_run.get_output(return_onnx_model=True)

#OnnxConverter.save_onnx_model(automl_fitted_model, './outputs/Automl_Onnex_Model.onnx')

In [ ]:

#model = best_run.register_model(model_name='best_autoML_run', model_path='outputs/Automl_Model.joblib')

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python
####  Workflow for deploying a model 
The workflow is similar no matter where you deploy your model:

<li>Register the model.
<li>Prepare an entry script.
<li>Prepare an inference configuration.
<li>Deploy the model locally to ensure everything works.
<li>Choose a compute target.
<li>Deploy the model to the cloud.
<li>Test the resulting web service.

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

##### Register Model

In [25]:
from azureml.core import Model

# Register model
automl_best_run.register_model(model_path='outputs/Maternal_Health_Risk_AutoML_model.pkl', 
                               model_name='automl_Maternal_Health_Risk_model_pickled',
                        tags={'Training context':'Maternal Health Risk Auto ML Model Trained'},
                        properties={'AUC': automl_best_run_metrics['AUC_weighted'], 
                                    'Accuracy': automl_best_run_metrics['accuracy']})


#automl_best_run.register_model( model_name='automl_Maternal_Health_Risk_model')
                                                                                

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path automl_Maternal_Health_Risk_model in the set of files uploaded to the run: ['accuracy_table', 'automl_driver.py', 'confusion_matrix', 'explanation/34a241da/classes.interpret.json', 'explanation/34a241da/expected_values.interpret.json', 'explanation/34a241da/features.interpret.json', 'explanation/34a241da/global_names/0.interpret.json', 'explanation/34a241da/global_rank/0.interpret.json', 'explanation/34a241da/global_values/0.interpret.json', 'explanation/34a241da/local_importance_values.interpret.json', 'explanation/34a241da/per_class_names/0.interpret.json', 'explanation/34a241da/per_class_rank/0.interpret.json', 'explanation/34a241da/per_class_values/0.interpret.json', 'explanation/34a241da/rich_metadata.interpret.json', 'explanation/34a241da/true_ys_viz.interpret.json', 'explanation/34a241da/visualization_dict.interpret.json', 'explanation/34a241da/ys_pred_proba_viz.interpret.j

####  List the registered model

In [ ]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

#### Prepare an inference configuration.

In [ ]:
from azureml.automl.core.shared import constants
automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'project_environment.yml')

#### Prepare an inference configuration using an entry script.

In [ ]:
from azureml.core.model import InferenceConfig
#from azureml.core.environment import Environment
from azureml.core import Environment

env = Environment(name="project_environment")
inference_config = InferenceConfig(entry_script="score.py",environment=env)

#inference_config = InferenceConfig(
#    environment=env,
#    source_directory="./source_dir",
#    entry_script="./echo_score.py",
#)

#### Deploy the model locally to ensure everything works. Will skip this part

In [4]:
#from azureml.core.webservice import LocalWebservice
#deployment_config = LocalWebservice.deploy_configuration(port=6789)
##Deploy your machine learning model
#service = Model.deploy(
#    ws,
#    "myservice",
#    [model],
#    dummy_inference_config,
#    deployment_config,
#    overwrite=True,
#)
#service.wait_for_deployment(show_output=True)
#print(service.get_logs())

In [ ]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

service=Model.deploy(workspace=ws,
                    name="Maternal-Health-Risk-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)



TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

service.delete()
model.delete()

In [ ]:
#Define a dummy entry script

#The entry script receives data submitted to a deployed web service and passes it to the model.
#It then returns the model's response to the client. The script is specific to your model. 
#The entry script must understand the data that the model expects and returns.

#The two things you need to accomplish in your entry script are:

#Loading your model (using a function called init())
#Running your model on input data (using a function called run())
#For your initial deployment, use a dummy entry script that prints the data it receives.


import json


def init():
    print("This is init")


def run(data):
    test = json.loads(data)
    print(f"received data {test}")
    return f"test is {test}"

#Save this file as echo_score.py inside of a directory called source_dir. This dummy script returns the data you send to it, so it doesn't use the model. But it is useful for testing that the scoring script is running.

#The following example demonstrates how to create a minimal environment with no pip dependencies,
#using the dummy scoring script you defined above



#The following Python demonstrates how to create a local deployment configuration:





**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
